<a href="https://colab.research.google.com/github/Aayush360/Natural_langauge_processing/blob/main/Text_Generator_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# that can generate text related to describing hotels for the city of Mumbai

In [6]:
# notebook imports

In [7]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

import re

from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Embedding

In [8]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Reading the data 

[Data-Source](https://data.world/promptcloud/hotels-on-makemytrip-com)

In [9]:
data=pd.read_csv('hotel.csv')

In [10]:
data.head()

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,latitude,longitude,mmt_holidayiq_review_count,mmt_location_rating,mmt_review_count,mmt_review_rating,mmt_review_score,mmt_traveller_type_review_count,mmt_tripadvisor_count,pageurl,property_address,property_id,property_name,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id,location
0,Hardasji Ki Magri,Udaipur,India,2016-06-21,{{facility}},|Zion Home Stay is located in a city that sets...,1 star,NaN,{{value}},False,24.581788,73.667805,NaN,..,NaN,{{ratingCriteria.name}}{{ratingCriteria.value}},4.5,Families:{{ratingSummaryInfo.miscMap['family']...,6.0,http://hotelz.makemytrip.com/makemytrip/site/h...,"Zion, F-17, Haridas Ji Ki Magri, Near Bal Vina...",201302071713316867,Zion Home Stay,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,78ddf880bd7937d384ff278cc5b39d6e,POINT(73.667805 24.581788)
1,Near Nai Gaon,Udaipur,India,2016-06-21,{{facility}},| Araliayas Resorts is a 3 star hotel located ...,3 star,NaN,{{value}},False,24.544342,73.644242,NaN,..,NaN,{{ratingCriteria.name}}{{ratingCriteria.value}},4.5,Families:{{ratingSummaryInfo.miscMap['family']...,34.0,http://hotelz.makemytrip.com/makemytrip/site/h...,"Udaipur-Jhadol Road , Near Nai Gaon",201312041703404727,Araliayas Resorts,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,9f9f9cbb2f7df8089b63d5cdeb257944,POINT(73.644242 24.544342)
2,Near Bagore Ki Haveli,Udaipur,India,2016-06-21,{{facility}},|A 2 star property is located at 24 km from Ma...,2 star,NaN,{{value}},False,24.579942,73.681953,NaN,..,NaN,{{ratingCriteria.name}}{{ratingCriteria.value}},NaN,Families:{{ratingSummaryInfo.miscMap['family']...,16.0,http://hotelz.makemytrip.com/makemytrip/site/h...,"23/208 Gangaur Ghat Near Bagore Ki Haveli , Ne...",201407171324437682,Shri Udai Palace,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,NaN,makemytrip,NaN,NaN,b314bb7fa8bfb1ed306f517be21d729e,POINT(73.681953 24.579942)
3,Dabok,Udaipur,India,2016-06-21,Airport Transfer|Car rental|Conference Hall|Cu...,|SNP House Airport Hotel And Restaurant is loa...,1 star,//imghtlak.mmtcdn.com/images/hotels/2014071815...,NaN,False,24.622607,73.855194,0.0,NaN,NaN,NaN,NaN,Families:|Couples:|Business:|Solo:|Friends:,3.0,http://hotelz.makemytrip.com/makemytrip/site/h...,"Opp GITS, NH-76, Bamaniya Khet, Dabok , Dabok",201407181538181173,SNP House Airport Hotel And Restaurant,Hotel,NaN,2016-06-21 04:06:50 +0000,`standard,/5,NaN,makemytrip,NaN,NaN,e6f5bb3c2d76a78d978b9ceb0e31ec56,POINT(73.855194 24.622607)
4,East Udaipur,Udaipur,India,2016-06-21,{{facility}},| Hotel Pichola Haveli is situated in the beau...,2 star,NaN,{{value}},False,24.580100,73.682200,NaN,..,NaN,{{ratingCriteria.name}}{{ratingCriteria.value}},3.7,Families:{{ratingSummaryInfo.miscMap['family']...,139.0,http://hotelz.makemytrip.com/makemytrip/site/h...,"64 Ghangour Ghat, Opp Bagore Ki Haveli , East ...",201409021258573714,Hotel Pichola Haveli (LAKE SIDE),Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,3.7,makemytrip,NaN,NaN,63072c301427b6ca450d31eea127bcf0,POINT(73.6822 24.5801)


In [11]:
data.city.value_counts()

NewDelhiAndNCR    1163
Goa               1122
Mumbai             543
Jaipur             534
Bangalore          512
                  ... 
Forbesganj           1
ChambaHimachal       1
Ramtek               1
Vengurla             1
Etawah               1
Name: city, Length: 770, dtype: int64

A substantial amount of hotel data form the city of Mumabai is available (543). Lets get description for Mumbai Hotels

In [12]:
array=['Mumbai']
data_mum = data.loc[data['city'].isin(array)]

In [13]:
data_mum.head()

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,latitude,longitude,mmt_holidayiq_review_count,mmt_location_rating,mmt_review_count,mmt_review_rating,mmt_review_score,mmt_traveller_type_review_count,mmt_tripadvisor_count,pageurl,property_address,property_id,property_name,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id,location
294,Charai,Mumbai,India,2016-08-28,Doctor on Call|Front desk|Laundry Service|Park...,"Nestled in Mumbai, a city with strong historic...",3,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,False,19.198414,72.974173,NaN,NaN,0.0,NaN,0.0,Family:0|Couple:0|Solo:0|Friends:0|Business:0,2.0,NaN,"Edulji Road,Opp MTNL Exchange, Charai,Thane",201508201737182754,Hotel Divya International,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Ac Superior Room|A/c Standard Rooms Double Occ...,NaN,NaN,makemytrip,Maharashtra,NaN,d78fae90ef2e1b5c2dfd547c61763a25,POINT(72.974173 19.198414)
309,Andheri (East),Mumbai,India,2016-08-28,Air Conditioned|Airport Transfer|Conference Ha...,3 km from Chhatrapati Shivaji International Ai...,2,NaN,Bathroom Toiletries|Daily Newspaper|Kitchenett...,False,19.117263,72.882023,NaN,Location:3.2,85.0,Excellent:8|Very Good:14|Average:23|Poor:22|Te...,3.0,Family:20|Couple:13|Solo:21|Friends:4|Business:6,68.0,NaN,"Opp.N.G.Complex, Ashok Nagar,Military Road, Ma...",200709211017493525,Hotel Ace Residency,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Executive|Deluxe,NaN,NaN,makemytrip,Maharasta,Location:3.2/5 | Hospitality:3.1/5 | Facilitie...,030865f741982beb373efddecdc6d6c3,POINT(72.882023 19.117263)
321,Khar,Mumbai,India,2016-08-28,Airport/Rlwy Stn Transfer|Bar|Conference Hall|...,Location Hotel Royal Garden is situated on Juh...,3,NaN,Electronic Safe|Bathroom Toiletries|Daily News...,False,19.091400,72.827976,NaN,Location:4.5,170.0,Excellent:55|Very Good:23|Average:44|Poor:37|T...,3.6,Family:51|Couple:38|Solo:44|Friends:7|Business:12,72.0,NaN,"Juhu Tara Road,Juhu, Near Juhu Beach,Western S...",201101101624406782,Hotel Royal Garden,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Superior Executive,NaN,NaN,makemytrip,Maharashtra,Location:4.5/5 | Hospitality:3.4/5 | Facilitie...,a1ced509350038775a7700ec67796bc2,POINT(72.827976 19.0914)
334,Andheri (East),Mumbai,India,2016-08-28,24 Hour Check in-Icon|24 hour reception|24 hou...,City Guest House is a beautiful property locat...,2,NaN,Bathroom Toiletries|Hot/cold Water|Attached Ba...,False,19.016200,72.843100,NaN,Location:4.3,42.0,Excellent:19|Very Good:5|Average:4|Poor:14|Ter...,3.8,Family:18|Couple:11|Solo:10|Friends:0|Business:0,5.0,NaN,"3, Framroz Court, 1st Floor,dadasaheb phalke r...",201508171552242376,City Guest House,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Standard Room|Deluxe Room|Triple Deluxe Room|S...,NaN,NaN,makemytrip,Maharashtra,Location:4.3/5 | Hospitality:3.8/5 | Facilitie...,f2820ae7707668ed6906bb227921f720,POINT(72.8431 19.0162)
1238,Andheri (East),Mumbai,NaN,2016-08-22,24 Hour Check in-Icon|24 hour reception|24 hou...,Sai Residency Hotel is situated in the City of...,2,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,False,19.109194,72.871042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"marol pipe line ,opp mukund hospital next to s...",201603281410379322,SaiResidency Hotel,Hotel,2016-08-22 22:10:53 +0000,2016-08-22 22:10:53 +0000,Deluxe Dbl Air Cooled,NaN,NaN,makemytrip,MAHARASHTRA,NaN,b4af24952027ffbcd85a91bb6fe23f5d,POINT(72.87104239999996 19.1091936)


Since we are interested in generating hotel description we will only keep the hotel overview column, since other columns will not be required for our analysis

Also let us remove all the descriptions that are empty

In [14]:
data_mum = data_mum.hotel_overview
data_mum = data_mum.dropna()

In [15]:
data_mum.head() # we have series of reviwes about Mumbai hotels

294     Nestled in Mumbai, a city with strong historic...
309     3 km from Chhatrapati Shivaji International Ai...
321     Location Hotel Royal Garden is situated on Juh...
334     City Guest House is a beautiful property locat...
1238    Sai Residency Hotel is situated in the City of...
Name: hotel_overview, dtype: object

### Pre-Process the data

In [16]:
# perfrom case-folding (converting to lowercase, stopword removal, keeping only alphabetic data), Also discard single character word

In [17]:
stop = set(stopwords.words('english'))

def stopword_removal(data_point):
  data= [x for x in data_point.split() if x not in stop]
  return data


In [18]:
ex = 'Nestled in Mumbai, a city with strong historic'
ex = stopword_removal(ex)
print(ex)

['Nestled', 'Mumbai,', 'city', 'strong', 'historic']


In [19]:
# method for overall data cleansing

In [20]:
def clean_data(data): 
  '''works for single sentence and returns cleaned tokens of words in the sentence along with unique_words'''
  cleaned_data =[]
  all_unique_words_in_each_description=[]
  for entry in data:
    entry = re.sub(pattern='[^a-zA-Z]',repl=" ", string=entry) # check if alphabet, remove non-alphabet
    entry = re.sub(r'\b\w{0,1}\b', repl=" ", string=entry)  # remove single word or blank character/word
    entry =entry.lower()
    entry = stopword_removal(entry)
    cleaned_data.append(entry) # list with in list 0(1) time-complexity
    unique = list(set(entry))
    all_unique_words_in_each_description.extend(unique) # appended list O(K) time-complexity, k- lenght of list need to be added
  return cleaned_data,all_unique_words_in_each_description

In [21]:
# function to make vocabulary

def unique_words(data):
  unique_words = set(data)
  return unique_words, len(unique_words)

#### Applying data cleaning to our df_mum

In [22]:
cleaned_data, all_unique_words_in_each_description = clean_data(data_mum)

In [23]:
unique_words, len_unique_words = unique_words(all_unique_words_in_each_description)

In [24]:
len_unique_words  # this is our vocab_size

3395

In [25]:
## peeking at one of the cleaned data

len(cleaned_data[0])

136

### Build Mapping for Words

### word --> index and index --> word

In [26]:
def build_indices(unique_words):
  word_to_index = {}
  index_to_word = {}
  for i,word in enumerate(unique_words):
    word_to_index[word]=i
    index_to_word[i]=word
  return word_to_index, index_to_word

In [27]:
word_to_index, index_to_word = build_indices(unique_words)

### Prepare Training Corpus

In [28]:
def prepare_corpus(corpus,word_to_index):
  sequences= []
  for line in corpus:
    tokens = line
    for i in range(1, len(tokens)):
      i_gram_sequence = tokens[:i+1]
      i_gram_sequence_ids = []

      for j,token in enumerate(i_gram_sequence):
        i_gram_sequence_ids.append(word_to_index[token])
      sequences.append(i_gram_sequence_ids)
  return sequences

In [29]:
sequences=prepare_corpus(cleaned_data,word_to_index)
max_sequence_len = max([len(x) for x in sequences]) # find sentence with maximum length

In [30]:
len(sequences) # our sequences contains 51836 list with colleciton of n-grams for each sentence staritng from length 2

51836

In [31]:
max_sequence_len # so the max len of sentece after cleaning and tokenizing is 308

308

In [32]:
sequences[0] # looking at the first sentence and its i-gram indices (first sentence with 2 words indexes)

[285, 2838]

In [33]:
sequences[1]

[285, 2838, 474]

In [34]:
# let us see which words are mapped to these indices

print(index_to_word[3125])
print(index_to_word[66])
print(index_to_word[1864])

delved
delicacies
around


In [35]:
# Finally, we have correctly build our sequences

In [36]:
## Preparing the data for trainng the model
# we should pad the input to the longest words in the sequences 308 in this case
# also we need to split the data into dependent and independent variable

In [37]:
def build_input_data(sequences,max_sequence_len, len_unique_words):
  sequences = np.array(pad_sequences(sequences,maxlen=max_sequence_len, padding='pre'))
  X = sequences[:,:-1]
  y = sequences[:,-1]
  y = np_utils.to_categorical(y,len_unique_words) # convert correspoding row index to 1, and len of each word is vector of length len_unique_words
  return X,y


In [38]:
X,y = build_input_data(sequences,max_sequence_len,len_unique_words)

### Building The model

input_dim: This is the size of the vocabulary in the text data. For example, if your data is integer encoded to values between 0-10, then the size of the vocabulary would be 11 words.
output_dim: This is the size of the vector space in which words will be embedded. It defines the size of the output vectors from this layer for each word. For example, it could be 32 or 100 or even larger. Test different values for your problem.
input_length: This is the length of input sequences, as you would define for any input layer of a Keras model. For example, if all of your input documents are comprised of 1000 words, this would be 1000.

In [39]:
# 10 indicates that we want a Dense embedding of size 10 as output from our model.
# 128 LSTM units in the hidden layer, dimensionality of the inner cells in the LSTM layer.
# randomly drop off 20% of neurons from the network using the Dropout layer.
#  multi-class classification problem, and so the softmax activation function is used.

In [40]:
def create_model(max_sequence_len, len_unique_words):
  model = Sequential()
  model.add(Embedding(len_unique_words,10,input_length=max_sequence_len-1))
  model.add(LSTM(128))
  model.add(Dropout(0.2))
  model.add(Dense(len_unique_words, activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam')
  return model


In [41]:
model = create_model(max_sequence_len, len_unique_words)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 307, 10)           33950     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               71168     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 3395)              437955    
Total params: 543,073
Trainable params: 543,073
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X,y,batch_size=512, epochs=100)

### Testing and Text Generation

In [43]:
def generate_text(seed_text, next_words, model, max_seq_len):
  for _ in range(next_words):
    cleaned_data = clean_data([seed_text])
    sequences = prepare_corpus(cleaned_data[0],word_to_index)
    sequences = pad_sequences([sequences[-1]],maxlen=max_seq_len-1,padding='pre')
    predicted = model.predict_classes(sequences, verbose=0)
    output_word=''
    output_word = index_to_word[predicted[0]]
    seed_text = seed_text+" "+output_word
  return seed_text.title()

In [44]:
print(generate_text("mumbai is a city of beautiful",20,model,max_sequence_len))


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Mumbai Is A City Of Beautiful Residency Located Distance Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati Shivaji International Airport Km Chhatrapati


Hyperparameter tuning, along with building more complex and larger models, can help in generating better results.